<a href="https://colab.research.google.com/github/phersh/cleanlab-pinecone-RAG/blob/main/Copy_of_Cleanlab_Pinecone_Reliable_Improved_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to build a reliable, curated, and accurate RAG system using Cleanlab and Pinecone

Retrieval-Augmented Generation (RAG) is a powerful technique in natural language processing that combines the strengths of large language models with external knowledge retrieval. RAG systems enhance the capabilities of AI models by allowing them to access and utilize relevant information from a curated knowledge base, leading to more accurate, up-to-date, and context-aware responses. Building a reliable, curated, and accurate RAG system is crucial for several reasons:

- **Improved Accuracy:** By retrieving relevant information, RAG systems can provide more precise and factual responses.
- **Reduced Hallucinations:** Access to external knowledge helps minimize the risk of AI models generating false or inconsistent information.
- **Adaptability:** RAG systems can be easily updated with new information without retraining the entire model.
- **Trustworthiness:** The ability to trace responses back to source documents enhances explainability and trust in AI-generated content.

In this notebook, we'll explore how to build such a system using [Cleanlab](https://cleanlab.ai/) and Pinecone.
Cleanlab is a company that specializes in data-centric AI, focusing on improving the quality and reliability of AI/ML systems. Cleanlab's Trustworthy Language Model (TLM) is designed to add trust and reliability to AI model outputs and indicate when it's unsure of an answer, making it ideal for applications where unchecked hallucinations could be problematic. This is especially the case for anyone trying to build generative AI applications in production.

Pinecone, as a vector database solution, excels in storing and retrieving high-dimensional vectors, making it perfect for managing and querying large-scale document embeddings.

By combining Cleanlab's TLM with Pinecone's vector database capabilities, we can create a RAG system that not only retrieves relevant information efficiently but also ensures the quality and reliability of the data being used. This notebook will guide you through the process of:

1. Using Cleanlab's TLM to tag and clean document data, removing low-quality chunks and personally identifiable information (PII).
2. Leveraging Pinecone to create and manage a vector database for storing and retrieving document embeddings.
3. Implementing a RAG pipeline that uses both tools to provide accurate and trustworthy responses.
4. Utilizing Cleanlab's TLM to classify metadata, enhance retrieval, and evaluate the trustworthiness of RAG outputs.

By the end of this notebook, you'll have a robust understanding of how to build a RAG system that prioritizes reliability, curation, and accuracy, setting a strong foundation for developing trustworthy AI applications.


![Reliable RAG with Pinecone and Cleanlab](https://github.com/pinecone-io/examples/blob/master/learn/generation/better-rag/assets/cleanlab_pinecone_RAG.png?raw=1)

Let's install dependencies to run the notebook and then import our libraries:

In [1]:
!pip install pinecone==5.0.1 sentence-transformers==3.0.1 cleanlab-studio==2.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: Click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninst

In [2]:
import pandas as pd
import time
import warnings
import matplotlib.pyplot as plt
import os
import pinecone
import uuid
from pinecone import ServerlessSpec
from sentence_transformers import SentenceTransformer
from typing import List, Tuple, Dict, Optional
from difflib import SequenceMatcher
import re

warnings.filterwarnings('ignore')

In [3]:
if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = input("Please enter your Pinecone API key: ")

Please enter your Pinecone API key: pcsk_e4E1h_Tzz9ciWpjd7u4eBeECAqHQ91PHP53F2em8jW7Hh7MsAvvjq8ia7Yba7n1xRws4V


Below is an example of how to set a Python index serverless specification. This allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/guides/indexes/understanding-indexes).

In [5]:
cloud = os.environ.get("PINECONE_CLOUD") or "aws"
region = os.environ.get("PINECONE_REGION") or "us-east-1"

spec = ServerlessSpec(cloud=cloud, region=region)

## Fetch and Load Documents Data

First let's fetch the dataset.

In [6]:
!wget -nc https://cleanlab-public.s3.amazonaws.com/Datasets/documents-RAG-demo.csv

--2024-12-18 03:21:14--  https://cleanlab-public.s3.amazonaws.com/Datasets/documents-RAG-demo.csv
Resolving cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)... 52.217.112.121, 3.5.8.117, 54.231.168.65, ...
Connecting to cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)|52.217.112.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1198031 (1.1M) [text/csv]
Saving to: ‘documents-RAG-demo.csv’

documents-RAG-demo. 100%[===================>]   1.14M  --.-KB/s    in 0.1s    

2024-12-18 03:21:14 (8.83 MB/s) - ‘documents-RAG-demo.csv’ saved [1198031/1198031]



Let's read in our documents data that we will use in this workflow. The data columns include the `filename` of the document and the document `text`.

In [7]:
# Read in dataset
df = pd.read_csv("documents-RAG-demo.csv")
print(df.shape)

(103, 3)


In [8]:
df.head()

,index,filename,text
0,0,documents/Blackstone-Third-Quarter-2023-Invest...,Blackstone Third Quarter 2023 Investor Call Oc...
1,1,documents/8k-nike.pdf,SECURITIES AND EXCHANGE COMMISSIONFORM 8-K Cur...
2,2,documents/FY24-Q1-NIKE-Press-Release.pdf,NaN
3,3,documents/10-K 2022-Apple2.pdf,The future principal payments for the Company’...
4,4,documents/q3-fy22-earnings-presentation.pdf,Financial\tpresentation\tto\t accompany\tmanag...


## Use Cleanlab's TLM to tag your documents with a topic

For our RAG system, it would be ideal to tag each of our documents with a particular topic that is most relevant (based on the text content), which can be used as [metadata](https://docs.pinecone.io/guides/data/filter-with-metadata) we can filter with during retrieval in our RAG system.

We can now use Cleanlab's [Trustworthy Language Model](https://cleanlab.ai/blog/trustworthy-language-model/) to tag our document chunks with the correct document topic that we can later use to enhance our retrieval process of fetching the correct context from our vector DB.

Cleanlab's Trustworthy Language Model (TLM) is a more reliable LLM that gives high-quality outputs and indicates when it is unsure of the answer to a question, making it suitable for applications where unchecked hallucinations are a show-stopper.

### Installing Cleanlab TLM

Using TLM requires a [Cleanlab](https://app.cleanlab.ai/) account. Sign up for one [here](https://cleanlab.ai/signup/) if you haven't yet. If you've already signed up, check your email for a personal login link.

Let's initialize the TLM client. Here we use powerful TLM settings, but check out the [TLM quickstart tutorial](https://help.cleanlab.ai/tlm/tutorials/tlm/) for configuration options to get results tailored to your use case.


## Using the TLM

You can use the TLM pretty much like any other LLM API:

In [ ]:
from cleanlab_studio import Studio

key = input("Please enter your Cleanlab API key: ")

# Get API key from here: https://app.cleanlab.ai/account after creating an account.
studio = Studio(key)

# Instantiate TLM
tlm = studio.TLM()

# Prompt the user to enter their own prompt
user_prompt = input("Enter your prompt: ")

output = tlm.prompt(user_prompt)

print(f"Output: {output}")

Let's now use Cleanlab's TLM to do classification and classify the text (tag) into different topics. We will make use of code from [Cleanlab's TLM Zero-Shot Classification Tutorial](https://help.cleanlab.ai/tutorials/zero_shot_classification/) to do this. This includes the two helper functions,`parse_category()` and `classify()`, that can be found below.

In [ ]:
# Helper Functions

def parse_category(
    response: str,
    categories: List[str],
    disable_warnings: bool = False
) -> str:
    """
    Extracts one of the provided categories from the response using regex patterns.

    If no category out of the possible categories is directly mentioned in the response,
    the category with greatest string similarity to the response is returned (along with a warning).

    Args:
        response (str): Response from the LLM
        categories (List[str]): List of expected categories
        disable_warnings (bool): If True, print warnings are disabled

    Returns:
        str: The extracted or best-matching category
    """
    response_str = str(response)
    escaped_categories = [re.escape(output) for output in categories]
    categories_pattern = "(" + "|".join(escaped_categories) + ")"

    exact_matches = re.findall(categories_pattern, response_str, re.IGNORECASE)
    if len(exact_matches) > 0:
        return str(exact_matches[-1])

    best_match = max(
        categories, key=lambda x: SequenceMatcher(None, response_str, x).ratio()
    )
    similarity_score = SequenceMatcher(None, response_str, best_match).ratio()

    if similarity_score < 0.5:
        warning_message = f"None of the categories remotely match raw LLM output: {response_str}.\nReturning the last entry in the constrain outputs list."
        best_match = categories[-1]
    else:
        warning_message = f"None of the categories match raw LLM output: {response_str}"

    if not disable_warnings:
        warnings.warn(warning_message)

    return best_match

def classify(texts: List[str], categories: List[str], prompt_template: str) -> Tuple[List[str], List[float]]:
    """
    Classifies a list of texts into predefined categories using a language model.

    Args:
        texts (List[str]): List of texts to classify
        categories (List[str]): List of possible categories
        prompt_template (str): Template string for formatting the prompt

    Returns:
        Tuple[List[str], List[float]]: A tuple containing two lists:
            - List of predicted categories for each text
            - List of trustworthiness scores for each prediction
    """
    prompts = [prompt_template.format(text=text) for text in texts]
    outputs = tlm.prompt(prompts)

    responses = [output['response'] for output in outputs]
    trustworthiness_scores = [output['trustworthiness_score'] for output in outputs]

    predictions = [parse_category(response, categories) for response in responses]

    return predictions, trustworthiness_scores


Now we can define the prompt and categories we will use to tag our documents with the correct topic.

In [ ]:
# Use Cleanlab's TLM to tag your documents with a topic
tagging_prompt = """
You are an assistant for tagging text as one of several topics. The available topics are:

1. 'finance': Related to financial matters, budgeting, accounting, investments, lending, or monetary policies.
2. 'hr': Pertaining to Human Resources, including hiring, employee documents (such as a W4 form), employee management, benefits, or workplace policies.
3. 'it': Covering Information Technology topics such as software development, network infrastructure, cybersecurity, or tech support.
4. 'product': Dealing with a specific company product, product development, management, features, or lifecycle.
5. 'sales': Involving selling a product, customer acquisition, revenue generation, or sales performance.

If you are not sure which topic to tag the text with, then answer 'unknown'. Only use the lower case text version of the topic name.

Task: Analyze the following text and determine the topic it belongs to. Return the topic as a string.

Now here is the Text to verify:

Text: {text}

Topic:
"""

categories = ['finance', 'hr', 'it', 'product', 'sales', 'unknown']

Next we will use our helper functions to tag our documents with the correct topic.

In [ ]:
predictions, trustworthiness_scores = classify(df["text"].tolist(), categories, tagging_prompt)

topics_df = df.copy()
topics_df['topic'] = predictions
topics_df['topic_trustworthiness'] = trustworthiness_scores

Querying TLM... 100%|██████████|


In [ ]:
# Display results
topics_df.head()

,index,filename,text,topic,topic_trustworthiness
0,0,documents/Blackstone-Third-Quarter-2023-Invest...,Blackstone Third Quarter 2023 Investor Call Oc...,finance,0.995676
1,1,documents/8k-nike.pdf,SECURITIES AND EXCHANGE COMMISSIONFORM 8-K Cur...,finance,0.995666
2,2,documents/FY24-Q1-NIKE-Press-Release.pdf,NaN,unknown,0.962286
3,3,documents/10-K 2022-Apple2.pdf,The future principal payments for the Company’...,hr,0.263197
4,4,documents/q3-fy22-earnings-presentation.pdf,Financial\tpresentation\tto\t accompany\tmanag...,finance,0.962706


As seen above, after running `classify()`, you will notice two new columns in the dataset:

- `topic`, a column with the response that we prompted for
- `topic_trustworthiness`, a corresponding trustworthiness score, which quantifies how confident you can be that the response is correct

Now we can use the [Trustworthiness Score](https://help.cleanlab.ai/tutorials/tlm/#how-does-the-tlm-trustworthiness-score-work) obtained in our results (as `topic_trustworthiness`) to analyze which of our topic responses are the most trustworthy and least trustworthy.

For our use case, at a low enough threshold score for `topic_trustworthiness`, we can replace the `topic` value for those responses with `unknown` since we can't be sure to trust those responses.

This threshold can be determined by sorting the results by the trustworthiness score and then looking for a cutoff point in the results when topic values seem untrustworthy.

In practice, generally speaking, if you have the time/resources, your team can manually review low-trustworthiness responses and provide a better human response instead. If not, you can determine a trustworthiness threshold below which responses seem untrustworthy, and automatically append a warning statement to any response whose trustworthiness falls below the threshold or flag those responses in the appropriate way for your use case.

The overall magnitude/range of the trustworthiness scores may differ between datasets, so we recommend selecting any thresholds to be application-specific. First consider the relative trustworthiness levels between different data points before considering the overall magnitude of these scores for individual data points.

In [ ]:
sorted_topic_df = topics_df.sort_values(
    by="topic_trustworthiness", ascending=False
).copy()
sorted_topic_df.head(10)

,index,filename,text,topic,topic_trustworthiness
7,7,documents/10-K 2022-Apple.pdf,Apple Inc.CONSOLIDATED STATEMENTS OF SHAREHOLD...,finance,0.999823
13,13,documents/Walmart_2022_Investor_Information.pdf,Corporate and Stock InformationListing New Yor...,finance,0.999793
12,12,documents/ir-policy_sept-2021.pdf,INVESTOR RELATIONS POLICYINVESTOR RELATIONS1IN...,finance,0.999789
9,9,documents/IBM_2Q_2017_10-Q.pdf,Table of ContentsUNITED STATES SECURITIES AND ...,finance,0.999772
36,36,documents/managers-new-employee-onboarding-gui...,Manager’s New Employee Onboarding GuideWelcome...,hr,0.999741
6,6,documents/DK_DB_InvestorPres_FINAL.ppt,Dara Khosrowshahi EVP & Chief Financial Office...,finance,0.999723
60,60,documents/Technology-Use-Permission.pdf,Please return completed form at Back to School...,it,0.999592
45,45,documents/2012_14.doc,"Calendar Year 2012 September 12, 2012 Volume 2...",hr,0.999582
54,54,documents/Computer-Equipment-Request.pdf,COMPUTER EQUIPMENT REQUEST FORM Please use thi...,it,0.999550
18,18,documents/Form-10-Q-Servicenow.pdf,Table of ContentsUNITED STATESSECURITIES AND E...,finance,0.999476


In [ ]:
sorted_topic_df.tail(10)

,index,filename,text,topic,topic_trustworthiness
8,8,documents/LD_Trucost_Company_Presentation_0504...,Quantitative Environmental Performance Measure...,unknown,0.705944
59,59,documents/internetsafety.ppt,Santa Rosa District SchoolsINTERNET SAFETYNove...,unknown,0.673393
61,61,documents/eligible-services.ppt,Eligible Services Service Provider TrainingS...,unknown,0.671809
63,63,documents/TecNewEquipmentRequest.pdf,Grade Level/Dept StaffYes Cost: ______________...,unknown,0.618944
35,35,documents/Research-Assistant.pdf,Job Description and Responsibilities forResear...,unknown,0.598968
20,20,documents/UnreleasedGames.doc,EVERY GAME THAT WAS NEVER RELEASED FOR THE SPE...,unknown,0.581846
42,42,documents/HR-Documents-Quick-Start-Guide.pdf,GreenEmployee.com HR DocumentsThis quick start...,unknown,0.566623
27,27,documents/Corrective-Action-Disciplinary-Actio...,Revision February 2008\nGuideline for BPPM 60....,unknown,0.354750
3,3,documents/10-K 2022-Apple2.pdf,The future principal payments for the Company’...,hr,0.263197
79,79,documents/specs-verticalprintinspectors.pdf,Specifications - Vertical Print InspectorsMode...,unknown,0.262263


After sorting the results in descending order by the `topic_trustworthiness` score, the results seem to be less trustworthy for scores less than 0.8.

Let's replace each of these less trustworthy`topic` responses with `unknown` now.

In [ ]:
topics_df.loc[topics_df["topic_trustworthiness"] < 0.8, "topic"] = (
    "unknown"
)

To get a sense of the distribution of topics we've tagged in our documents data, let's look at the distribution now:

In [ ]:
topic_counts = topics_df["topic"].value_counts(dropna=False)
print(f"\nTopic Column Distribution:\n{topic_counts}")

# Percentage distribution of topics
topic_percentages = topics_df["topic"].value_counts(normalize=True) * 100
print(f"\nTopic Percentage Distribution:\n{topic_percentages}")


Topic Column Distribution:
topic
hr         27
unknown    21
product    18
finance    17
sales      16
it          4
Name: count, dtype: int64

Topic Percentage Distribution:
topic
hr         26.213592
unknown    20.388350
product    17.475728
finance    16.504854
sales      15.533981
it          3.883495
Name: proportion, dtype: float64


## Initialize RAG Pipeline using our documents data

In [ ]:
class PineconeRAGPipeline:
    def __init__(
        self,
        model_name: str = "paraphrase-MiniLM-L6-v2",
        index_name: str = "document-index",
        cloud: str = "aws",
        region: str = "us-east-1",
    ):
        """
        Initialize the PineconeRAGPipeline with a specified model and index name.

        Args:
            model_name (str): Name of the SentenceTransformer model to use.
            index_name (str): Name of the Pinecone index to create or connect to.
            cloud (str): Cloud provider for Pinecone.
            region (str): Region for the Pinecone service.
        """
        self.model = SentenceTransformer(model_name)
        if not os.environ.get("PINECONE_API_KEY"):
            os.environ["PINECONE_API_KEY"] = "YOUR PINECONE API KEY HERE"
        self.pc = pinecone.Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
        self.index_name = index_name

        existing_indexes = self.pc.list_indexes()

        if self.index_name not in existing_indexes:
            try:
                print(f"Creating new index: {self.index_name}")
                self.pc.create_index(
                    name=self.index_name,
                    dimension=self.model.get_sentence_embedding_dimension(),
                    metric="cosine",
                    spec=pinecone.ServerlessSpec(cloud=cloud, region=region),
                )
            except Exception as e:
                if "ALREADY_EXISTS" in str(e):
                    print(
                        f"Index {self.index_name} already exists. Connecting to existing index."
                    )
                else:
                    raise e
        else:
            print(
                f"Index {self.index_name} already exists. Connecting to existing index."
            )

        self.index = self.pc.Index(self.index_name)

    def chunk_text(self, text: str, max_tokens: int = 256) -> List[str]:
        """
        Split text into chunks based on a maximum token size.

        Args:
            text (str): The document text to be chunked.
            max_tokens (int): The maximum number of tokens per chunk.

        Returns:
            List[str]: List of text chunks.
        """
        words = text.split()
        chunks = []
        current_chunk = []
        current_chunk_tokens = 0

        for word in words:
            word_tokens = len(self.model.tokenize([word])["input_ids"][0])
            if current_chunk_tokens + word_tokens > max_tokens and current_chunk:
                chunks.append(" ".join(current_chunk))
                current_chunk = []
                current_chunk_tokens = 0

            current_chunk.append(word)
            current_chunk_tokens += word_tokens

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        for i, chunk in enumerate(chunks):
            print(
                f"Chunk {i+1} length: {len(chunk)} characters, "
                f"{len(self.model.tokenize([chunk])['input_ids'][0])} tokens"
            )

        return chunks

    def index_documents(self, df: pd.DataFrame) -> int:
        """
        Index documents from a DataFrame with specific metadata structure.

        Args:
            df (pd.DataFrame): DataFrame containing document information and metadata.
                               Expected columns: 'text', 'filename', 'topic'

        Returns:
            int: The number of chunks successfully indexed.
        """
        valid_docs = []
        valid_metadata = []
        generated_ids = []

        print("Starting document processing...")

        for idx, row in df.iterrows():
            if pd.isna(row["text"]) or pd.isna(row["filename"]) or pd.isna(row["topic"]):
                print(f"Skipping invalid document at index {idx}: {row['filename']}")
                continue

            doc = str(row["text"])
            print(f"Processing document {row['filename']} at index {idx}...")

            chunks = self.chunk_text(doc)

            for i, chunk in enumerate(chunks):
                chunk_id = str(uuid.uuid4())
                chunk_metadata = {
                    "filename": row["filename"],
                    "topic": row["topic"],
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "chunk_id": chunk_id,
                }
                valid_docs.append(chunk)
                valid_metadata.append(chunk_metadata)
                generated_ids.append(chunk_id)

        print(f"Total chunks to encode: {len(valid_docs)}")

        if not valid_docs:
            print("No valid documents to index.")
            return 0

        doc_embeddings = self.model.encode(valid_docs)

        batch_size = 100
        for i in range(0, len(valid_docs), batch_size):
            batch_docs = valid_docs[i : i + batch_size]
            batch_metadata = valid_metadata[i : i + batch_size]
            batch_embeddings = doc_embeddings[i : i + batch_size]

            vectors = [
                (
                    generated_ids[i + j],
                    embedding.tolist(),
                    {**metadata, "text": doc[:1000]},
                )
                for j, (doc, embedding, metadata) in enumerate(
                    zip(batch_docs, batch_embeddings, batch_metadata)
                )
            ]

            try:
                self.index.upsert(vectors=vectors)
                print(f"Successfully indexed batch of {len(vectors)} chunks.")
            except Exception as e:
                print(f"Error during upsert: {e}")

        print("Document indexing completed.")
        return len(valid_docs)

    def search(
        self, query: str, top_k: int = 5, filter_query: Optional[Dict] = None
    ) -> List[Tuple[str, Dict]]:
        """
        Search for the top_k most relevant documents based on the input query and optional filter.

        Args:
            query (str): The search query text.
            top_k (int): The number of top relevant documents to return.
            filter_query (Optional[Dict]): Optional filter query to apply during search.

        Returns:
            List[Tuple[str, Dict]]: List of top_k relevant document texts and their metadata.
                                    Each tuple contains (document_text, metadata_dict).
        """
        query_embedding = self.model.encode(query)

        try:
            results = self.index.query(
                vector=query_embedding.tolist(),
                top_k=top_k,
                filter=filter_query,
                include_metadata=True,
            )

            return [
                (
                    match.metadata["text"],
                    {k: v for k, v in match.metadata.items() if k != "text"},
                )
                for match in results.matches
            ]
        except Exception as e:
            print(f"Error during search: {e}")
            return []

    def delete_index(self) -> None:
        """
        Delete the Pinecone index.

        Raises:
            Exception: If there's an error during the deletion process.
        """
        try:
            self.pc.delete_index(self.index_name)
            print(f"Index '{self.index_name}' has been deleted.")
        except Exception as e:
            print(f"Error deleting index: {e}")

    def extract_all_chunks_from_index(self, max_chunks: int = 10000) -> pd.DataFrame:
        """
        Extract all document chunks and metadata from the Pinecone index into a DataFrame.

        Args:
            max_chunks (int): Maximum number of chunks to retrieve.

        Returns:
            pd.DataFrame: DataFrame containing chunk data and metadata.
                          Columns include all metadata fields stored in the index.

        Raises:
            Exception: If there's an error retrieving chunks from the index.
        """
        stats = self.index.describe_index_stats()
        total_vectors = stats.total_vector_count
        dimension = stats.dimension

        print(f"Index name: {self.index_name}")
        print(f"Total vectors according to stats: {total_vectors}")
        print(f"Vector dimension: {dimension}")

        try:
            results = self.index.query(
                vector=[0.0] * dimension,
                top_k=max_chunks,
                include_values=False,
                include_metadata=True,
            )

            chunk_data = [match.metadata for match in results.matches]
            chunk_df = pd.DataFrame(chunk_data)

            print(f"Total chunks retrieved: {len(chunk_df)}")
            return chunk_df

        except Exception as e:
            print(f"Error retrieving chunks from index: {e}")
            return pd.DataFrame()

    def delete_chunks(self, chunk_ids: List[str]) -> None:
        """
        Delete specific chunks from the Pinecone index.

        Args:
            chunk_ids (List[str]): List of chunk IDs to delete.

        Raises:
            Exception: If there's an error during the deletion process.
        """
        try:
            self.index.delete(ids=chunk_ids)
            print(f"Successfully deleted {len(chunk_ids)} chunks from the index.")
        except Exception as e:
            print(f"Error deleting chunks: {e}")
            print(f"Problematic chunk IDs: {chunk_ids}")

        print(f"Finished deletion process for {len(chunk_ids)} chunks.")

Let's use the RAG pipeline we have defined above to create a Pinecone index - then we upsert our document chunks into our vector DB in batches.

In [ ]:
# Create RAG pipeline on unfiltered documents
rag_pipeline = PineconeRAGPipeline(index_name="cleanlab-pinecone-tutorial-index")
rag_pipeline.index_documents(topics_df)

After upserting our document chunks, let's now confirm that the upsertion worked properly (after waiting 30 seconds to give the vector DB time to update) by reading the document chunks into a new DataFrame `chunk_df`.

In [ ]:
time.sleep(30)

# Extract chunks from the Pinecone index into a DataFrame
chunk_df = rag_pipeline.extract_all_chunks_from_index()

# Display the resulting DataFrame
chunk_df.head()

Index name: cleanlab-pinecone-tutorial-index
Total vectors according to stats: 2432
Vector dimension: 384
Total chunks retrieved: 2432


,chunk_id,chunk_index,filename,text,topic,total_chunks
0,3078a09c-799c-474e-85e7-a9892411e76c,4.0,documents/Walmart_2022_Investor_Information.pdf,follows:2023HighLow1st Quarter(1)$146.94$132.0...,finance,9.0
1,4b21ab0f-01f2-48a3-a307-b2e3d90177e6,5.0,documents/2012_14.doc,Pay transactions on HUE01 using the following ...,hr,17.0
2,c30ad6e9-6b96-44ba-8fe9-07a69155c5ec,5.0,documents/internet_safety.pptx,"751-5980 (800) 487-1626 (8 a.m. to 5 p.m. CST,...",unknown,6.0
3,f5ab2c30-0ac6-4ae4-b22f-3c3c50de5a6c,139.0,documents/Investor_Transcript_2023-10-26.pdf,"PROVIDE AN ACCURATE TRANSCRIPTION, THERE MAY B...",finance,141.0
4,7b089b6f-30a4-474f-ac7c-c91ba7143728,12.0,documents/2012_14.doc,When entering deduction override amounts for a...,hr,17.0


In [ ]:
chunk_df.shape

(2432, 6)

In [ ]:
document_data_columns = list(chunk_df.columns)
print(f"The columns in our documents data are: {document_data_columns}")

The columns in our documents data are: ['chunk_id', 'chunk_index', 'filename', 'text', 'topic', 'total_chunks']


We can see below that the number of chunks per document varies:

In [ ]:
chunks_per_doc = chunk_df.groupby("filename")["chunk_index"].max() + 1
print(chunks_per_doc)

filename
documents/10-K 2022-Apple.pdf                                               3.0
documents/10-K 2022-Apple2.pdf                                             16.0
documents/2012_14.doc                                                      17.0
documents/2020_-_I-9_Employment_Verification_Training_Presentation.ppt     17.0
documents/2023-CEASampleEmployeeHandbook.doc                              286.0
                                                                          ...  
documents/the-selling-process-1.pdf                                         7.0
documents/the-selling-process-2.pdf                                        10.0
documents/unit04.pdf                                                       22.0
documents/what-is-selling.pptx                                              7.0
documents/work-ez-specs.pdf                                                 2.0
Name: chunk_index, Length: 95, dtype: float64


## Use Cleanlab's TLM to detect and filter out bad document chunks

Now that we have extracted our document chunks from our Pinecone index, let's use Cleanlab's TLM (via the helper functions we defined earlier to tag documents) to detect low quality document chunks (i.e. HTML, not interpretable/cut off phrases, or non English text) and personally identifiable information (PII) to filter out these chunks from our vector DB.

Below we define the prompts we will use to find the document chunks that are badly chunked or contain PII and run our `classify()` helper function to obtain the response and trustworthiness scores from TLM.

In [ ]:
# Bad chunks detection (you can replace this with your own definitions)
bad_chunks_prompt = """
I am chunking documents into smaller pieces to create a knowledge base for question-answering systems.

Task: Help me check if the following Text is badly chunked. A badly chunked text is any text that is: full of HTML/XML and other non-language strings or non-english words, has hardly any informative content or missing key information, or text that contains Personally-Identifiable Information (PII) and other sensitive confidential information.

Return 'bad_chunk' if the provided Text is badly chunked, and 'good_chunk' otherwise. Please be as accurate as possible, the world depends on it.

Text: {text}
"""

bad_chunk_categories = ["bad_chunk", "good_chunk"]

In [ ]:
bad_chunk_predictions, bad_chunk_scores = classify(chunk_df["text"].tolist(), bad_chunk_categories, bad_chunks_prompt)

chunk_df['chunk_quality'] = bad_chunk_predictions
chunk_df['chunk_quality_trustworthiness'] = bad_chunk_scores

Querying TLM... 100%|██████████|


In [ ]:
# PII detection
pii_prompt = """
I am chunking documents into smaller pieces to create a knowledge base for question-answering systems.

Task: Analyze the following text and determine if it has personally identifiable information (PII). PII is information that could be used to identify an individual or is otherwise sensitive. Names, addresses, phone numbers are examples of common PII.

Return 'is_PII' if the text contains PII and 'no_PII' if it does not. Please be as accurate as possible, the world depends on it.

Text: {text}
"""

pii_categories = ["is_PII", "no_PII"]

In [ ]:
pii_predictions, pii_scores = classify(chunk_df["text"].tolist(), pii_categories, pii_prompt)

chunk_df['pii_check'] = pii_predictions
chunk_df['pii_check_trustworthiness'] = pii_scores

Querying TLM...   0%|          |

Querying TLM... 100%|██████████|


In [ ]:
sorted_chunk_quality_df = chunk_df.sort_values(
    by="chunk_quality_trustworthiness", ascending=False
).copy()
sorted_is_pii_df = chunk_df.sort_values(
    by="pii_check_trustworthiness", ascending=False
).copy()

Now let's check how many document chunks are bad chunks or contain PII but also in which the response trustworthiness score is >= 0.95, which represent the most trustworthy responses using TLM.

In [ ]:
worst_chunks = sorted_chunk_quality_df.query(
    "chunk_quality == 'bad_chunk' and chunk_quality_trustworthiness >= 0.95"
)
worst_pii = sorted_is_pii_df.query(
    "pii_check == 'is_PII' and pii_check_trustworthiness >= 0.95"
)

print(
    f"Number of document chunks that have the worst chunk quality based on trustworthiness: {worst_chunks.shape[0]}"
)
print(
    f"Number of document chunks have the worst PII based on trustworthiness: {worst_pii.shape[0]}"
)

Number of document chunks that have the worst chunk quality based on trustworthiness: 87
Number of document chunks have the worst PII based on trustworthiness: 127


We have observed the capabilities that Cleanlab's TLM can provide in detecting these low quality document chunks, so let's now:

1. Update our Pinecone DB by removing the chunks with any of the issues detected by Cleanlab
2. Verify that the update to our Pinecone DB worked

Let's observe some of the worst chunks that we are going to delete:

In [ ]:
print(worst_chunks.iloc[0]["text"])
print("\n")
print(worst_chunks.iloc[1:2]["text"])

140,001 - 150,000 150,001 and over0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15$0 - $8,000 8,001 - 17,000 17,001 - 26,000 26,001 - 34,000 34,001 - 44,000 44,001 - 75,000 75,001 - 85,000 85,001 - 110,000 110,001 - 125,000 125,001 - 140,000 140,001 and over0 1 2 3 4 5 6 7 8 9 10$0 - $75,000 75,001


718    3, 5 2019 3, 5 2018 3, 4, 52,5522,3561,9621,77...
Name: text, dtype: object


These chunks are clearly not easy to understand nor do they represent full English phrases.

Now let's observe some of the worst examples of PII that we are going to delete:

In [ ]:
print(worst_pii.iloc[2:3]["text"])
print("\n")
print(worst_pii.iloc[6:7]["text"])

209    Cathy McGill at (804) 371-7800 or Email at cat...
Name: text, dtype: object


438    (TTY) (800) 700-2320; or visit the department’...
Name: text, dtype: object


These examples clearly contain sensitive information that is flagged by Cleanlab - so these definitely contain PII that can be removed!

We can now construct the list of chunks to delete based on the `chunk_id` we previously created for each chunk that we can tie to each chunk in our vector DB.

In [ ]:
# Get the chunk IDs for the chunks that have issues to update our Pinecone index
worst_chunks_to_delete_ids = worst_chunks["chunk_id"].tolist()
worst_pii_to_delete_ids = worst_pii["chunk_id"].tolist()
chunks_to_delete_ids = list(set(worst_chunks_to_delete_ids + worst_pii_to_delete_ids))

In [ ]:
len(chunks_to_delete_ids)

195

In [ ]:
# Delete the identified chunks from the index
rag_pipeline.delete_chunks(chunks_to_delete_ids)

Successfully deleted 195 chunks from the index.
Finished deletion process for 195 chunks.


After deleting our document chunks, let's now confirm that the deletion worked properly (after waiting 30 seconds to give the vector DB time to update) by reading the document chunks into a new DataFrame `updated_chunk_df`.

We will also then confirm the number of chunks deleted is equal to the total number of original chunks minus the number of chunks after filtering out bad chunks.

In [ ]:
# Sleep timer to allow the deletion process to complete
time.sleep(30)

# Verify the update went through and we have the correct number of chunks
updated_chunk_df = rag_pipeline.extract_all_chunks_from_index()
updated_chunk_df.head()

Index name: cleanlab-pinecone-tutorial-index
Total vectors according to stats: 2237
Vector dimension: 384
Total chunks retrieved: 2237


,chunk_id,chunk_index,filename,text,topic,total_chunks
0,f5ab2c30-0ac6-4ae4-b22f-3c3c50de5a6c,139.0,documents/Investor_Transcript_2023-10-26.pdf,"PROVIDE AN ACCURATE TRANSCRIPTION, THERE MAY B...",finance,141.0
1,3078a09c-799c-474e-85e7-a9892411e76c,4.0,documents/Walmart_2022_Investor_Information.pdf,follows:2023HighLow1st Quarter(1)$146.94$132.0...,finance,9.0
2,194cb427-9583-4858-bf58-5a5a92ea903c,0.0,documents/NHO 2015 - 2016_6-15.pps,Tennessee State UniversityNew EmployeeBenefits...,hr,25.0
3,4b21ab0f-01f2-48a3-a307-b2e3d90177e6,5.0,documents/2012_14.doc,Pay transactions on HUE01 using the following ...,hr,17.0
4,9caf3145-2e0b-4df9-80ec-57de1c28d10d,9.0,documents/Number_8.0_DISCIPLINARY_ACTION_AND_C...,be reviewed by the Department of Human Resourc...,hr,33.0


In [ ]:
# Confirm the number of chunks deleted is equal to the total number of original chunks minus the number of chunks after filtering out bad chunks
assert len(chunk_df) - len(updated_chunk_df) == len(chunks_to_delete_ids)

**Note:** You can also run this data ingestion (tagging, chunking, and cleaning) into a Pinecone vector DB index in real-time as new documents are being ingested.

### How to search for documents with metadata

Below is an example on how you can search for your curated documents based on a query of your choice and use metadata to help filter for the relevant information.

In this case, we specify `topic = sales` to find the `top k` (where k is number of results to return and equal to 2 here) documents that best match the search query/metadata filters.

In [ ]:
results = rag_pipeline.search(
    query="Tell me about sales",  # YOUR SEARCH QUERY HERE
    top_k=2,
    filter_query={"topic": {"$eq": "sales"}},
)

for doc_text, metadata in results:
    print(f"Document: {doc_text[:100]}...")
    print(f"Metadata: {metadata}")
    print("-" * 500)

Document: learn the person’s name, position, age, education, experience, hobbies, etc. Many times, just gettin...
Metadata: {'chunk_id': 'e9cb9756-2426-44e1-ada0-f50e7d8c17ae', 'chunk_index': 5.0, 'filename': 'documents/the-selling-process-2.pdf', 'topic': 'sales', 'total_chunks': 10.0}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Document: often used in a streamlined sales process.You have completed the topic for the sales process. Thank ...
Metadata: {'chunk_id': '62970e10-536e-4c0d-b925-6a632aebfb1b', 'chunk_index': 14.0, 'filename': '

## Use Cleanlab's TLM for intent classification to classify metadata

Suppose our user is asked which topic their question is about from a pre-defined list. Alternatively, we could train a classifier to predict the topic from the question (automatically using Cleanlab's TLM). This way, the intent classification is done automatically (and reliably) to obtain our topic that pertains to a user question. This topic can then be used for metadata filtering when querying against our vector DB.

In [ ]:
# Intent classification
intent_classification_prompt = """
You are an assistant for classifying a question as one of several topics. The available topics are:

1. 'finance': Related to financial matters, budgeting, accounting, investments, lending, or monetary policies.
2. 'hr': Pertaining to Human Resources, including hiring, employee documents (such as a W4 form), employee management, benefits, or workplace policies.
3. 'it': Covering Information Technology topics such as software development, network infrastructure, cybersecurity, or tech support.
4. 'product': Dealing with a specific company product, product development, management, features, or lifecycle.
5. 'sales': Involving selling a product, customer acquisition, revenue generation, or sales performance.

If you are not sure which topic to classify the question as, then answer 'unknown'.

Task: Analyze the following question and determine the topic it belongs to. Return the topic as a string.

Now here is the question to verify:

Text: {text}

Topic:
"""

In [ ]:
question = "What were Blackstone's fee-related earnings in the third quarter of 2023?"

Using the question and prompt defined above, let's now use TLM to classify the question into the correct topic (intent classification) that we will use as the metadata to filter our vector DB query with:

In [ ]:
intent_predictions, intent_trustworthiness_scores = classify([question], categories, intent_classification_prompt)

metadata_topic_response = {
    "metadata_response": intent_predictions[0],
    "metadata_trustworthiness_score": intent_trustworthiness_scores[0]
}

print(metadata_topic_response)

# Extract the topic from the response
metadata_topic = metadata_topic_response["metadata_response"]

Querying TLM... 100%|██████████|

{'metadata_response': 'finance', 'metadata_trustworthiness_score': 0.9059273817223932}


Below, you can then specify the classified metadata topic (or other relevant metadata) when filtering to help search for the relevant context in your Pinecone index.

Since we don't know where to stick the documents with an `unknown` topic value, we will always include the unknown topic in the filter.

In [ ]:
# Define the list of topics to filter on
topic_filter = [metadata_topic, "unknown"]

# Use the classified topic to filter the search results
top_doc_chunk = rag_pipeline.search(
    question, top_k=1, filter_query={"topic": {"$in": topic_filter}}
)
print(f"Top document chunk for this query: \n\n{top_doc_chunk[0][0]}")

Top document chunk for this query: 

of year-over-year base management fee growth at Blackstone. Fee-related earnings were $1.1 billion in the third quarter, or $0.92 per share, largely stable with Q2, underpinned by steady top-line performance, along with the firm's strong margin position. The year-over-year comparison was affected by a decline in transaction fees, which are activity-based, as well as lower fee-related performance revenues. Notwithstanding these headwinds, the firm generated $275 million of fee-related performance revenues in


## Use Cleanlab's TLM to get Trustworthiness Score for RAG Outputs

Now let's use the context from our top document chunk we obtained in the previous step to actually answer the original question asked: `What were Blackstone's fee-related earnings in the third quarter of 2023?`

In [ ]:
top_doc_chunks = "".join(
    rag_pipeline.search(
        question, top_k=1, filter_query={"topic": {"$in": topic_filter}}
    )[0][0]
)

prompt = f"""You are an assistant for answering the following question based on the document context.

Question: {question}
Document Context: {top_doc_chunks}
"""

Now let's query TLM using the prompt we previously created using our user query + document context passed from our RAG Pinecone index.

In [ ]:
output = tlm.prompt(prompt)
output

{'response': "Blackstone's fee-related earnings in the third quarter of 2023 were $1.1 billion, or $0.92 per share.",
 'trustworthiness_score': 0.95175686577078}

### Is this response correct?

In [ ]:
text_to_check = updated_chunk_df.query("filename == 'documents/Blackstone-Third-Quarter-2023-Investor-Call.pdf' and text.str.contains('of year-over-year base management fee', case=False, na=False)").iloc[0]["text"]
text_to_check[60:119]

'Fee-related earnings were $1.1 billion in the third quarter'

### Can we find a hallucination?

Let's now try to find a hallucination but first classify the intent of this new question into the correct metadata topic value.

In [ ]:
question = "Can you tell me if these Good's Homestyle Potato Chips support 9 per row in the case?"

In [ ]:
intent_predictions, intent_trustworthiness_scores = classify([question], categories, intent_classification_prompt)

metadata_topic_response = {
    "metadata_response": intent_predictions[0],
    "metadata_trustworthiness_score": intent_trustworthiness_scores[0]
}

print(metadata_topic_response)

# Extract the topic from the response
metadata_topic = metadata_topic_response["metadata_response"]

Querying TLM... 100%|██████████|

{'metadata_response': 'product', 'metadata_trustworthiness_score': 0.951910293755863}


Now we can pass the question and context into our prompt to catch this hallucination.

In [ ]:
# Define the list of topics to filter on
topic_filter = [metadata_topic, "unknown"]

top_doc_chunks = "".join(
    rag_pipeline.search(
        question, top_k=1, filter_query={"topic": {"$in": topic_filter}}
    )[0][0]
)

prompt = f"""You are an assistant for answering the following question based on the document context.

Question: {question}
Document Context: {top_doc_chunks}
"""

# Runs the Cleanlab TLM with confidence reliablity scores with default model and quality preset
tlm = studio.TLM()

output = tlm.prompt(prompt)
output

{'response': "Yes, the Good's Homestyle Potato Chips support 9 per row in the case, as indicated by the pallet configuration of 72 per pallet (9 x 8).",
 'trustworthiness_score': 0.7508270454814128}

### Is this correct?

There are 24 chip bags per case which cannot support 9 per row, so this is a hallucination.

In [ ]:
text_to_check = updated_chunk_df.query("filename == 'documents/potato-chips-specs.pdf'").iloc[0]["text"]
text_to_check[113:139]

'24/case (3 x 6 & 6 on top)'

### Flag less trustworthy responses with a warning for human review

You can boost the reliability of your Generative AI applications by adding contingency plans to override LLM answers whose trustworthiness score falls below some threshold (e.g., route to human for answer, append disclaimer that answer is uncertain, revert to a default baseline answer, or request a prompt with more information/context).

If you have time/resources, your team can manually review low-trustworthiness responses and provide a better human response instead.
If not, you can determine a trustworthiness threshold below which responses seem untrustworthy, and automatically append a warning statement to any response whose trustworthiness falls below the threshold.

The overall magnitude/range of the trustworthiness scores may differ between datasets, so we recommend selecting any thresholds to be application-specific. First consider the relative trustworthiness levels between different data points before considering the overall magnitude of these scores for individual data points. For more information on how to choose the appropriate threshold, you can refer to the [TLM quickstart tutorial](https://help.cleanlab.ai/tlm/tutorials/tlm/).

Since our hallucination has a trustworthiness score of `0.75` from our example above, we will set the threshold to `0.8` and append a warning statement to any response whose trustworthiness score falls below this threshold so that a human reviewer is aware of the potential untrustworthiness of the response.

In [ ]:
# Process the response based on trustworthiness threshold
threshold = 0.8  # set this after inspecting responses around different trustworthiness ranges
if output['trustworthiness_score'] < threshold:
    output['response'] = output['response'] + "\n CAUTION: THIS ANSWER HAS BEEN FLAGGED AS POTENTIALLY UNTRUSTWORTHY"

# Example using our previous output
print("Trustworthiness score:", output['trustworthiness_score'])
print("\nProcessed response with warning:", output['response'])

Trustworthiness score: 0.7508270454814128

Processed response with warning: Yes, the Good's Homestyle Potato Chips support 9 per row in the case, as indicated by the pallet configuration of 72 per pallet (9 x 8).
 CAUTION: THIS ANSWER HAS BEEN FLAGGED AS POTENTIALLY UNTRUSTWORTHY


### Apply Cleanlab's TLM to Existing RAG Prompt/Response Pairs

We can also make use of the [get_trustworthiness_score](https://help.cleanlab.ai/reference/python/trustworthy_language_model/#method-get_trustworthiness_score) function from Cleanlab's TLM Python API to generate a trustworthiness score for prompt/response pairs that you already have. This can be relevant for a RAG system in which you already have retrieved the context and obtained a response from your LLM but still want to add trust and reliability to these input/output pairs.

In [ ]:
# Redefine TLM to use the best quality preset and GPT-4o model
tlm = studio.TLM(quality_preset="best", options={"model": "gpt-4o"})

prompt = """Based on the following documents, answer the given question.

Documents:  We're getting new desks! Specs are here: Staples Model RTG120XLDBL BasePage \
CollectionModel | Dimensions width = 60.0in height = 48.0in depth = 24.0in Base Color Black \
Top Color White | Specs SheetsPowered by TCPDF (www.tcpdf.org)

Question: What is the width of the new desks?
"""

response_A = "60 inches"
response_B = "24 inches"

trust_score_A = tlm.get_trustworthiness_score(prompt, response_A)
trust_score_B = tlm.get_trustworthiness_score(prompt, response_B)

print(f"TLM Score for Response A: {trust_score_A}")
print(f"TLM Score for Response B: {trust_score_B}")

TLM Score for Response A: {'trustworthiness_score': 0.8271985087770306}
TLM Score for Response B: {'trustworthiness_score': 0.0635722930167351}


The trustworthiness score for the prompt/response pair that has the correct response is expectedly much higher (and therefore more trustworthy) than the prompt/response pair with the incorrect response.

To summarize, we used Cleanlab and Pinecone to build a reliable, curated, and accurate RAG system.

We used Pinecone to create the DB to actually store our document data and help us create accurate metadata to label our documents/chunks. We used Pinecone to create the vector database, actually store our chunks of document data/metadata, and as our similarity search engine - which all play a crucial part in the storage/retrieval components of RAG.

We used Cleanlab's Trustworthy Language Model (TLM) to first tag our document chunks and then ensure no badly chunked data or PII exists in the vector DB in order to curate more accurate and reliable documents. This ensures that, during retrieval from the RAG system, only the most relevant subset of  documents is considered. Lastly, we used TLM to classify metadata for use in retrieval and used TLM to show how to get trustworthiness scores for RAG Outputs to eliminate hallucinations and nonsensical answers.

For more guidance on how you can leverage TLM in RAG applications, you can refer to this Cleanlab Python API [tutorial](https://help.cleanlab.ai/tutorials/tlm_rag/).